# COT5600 Homework Assignment 4

Daniel Silva

In [0]:
import numpy as np

## Problem 1

### Floyd's collision detection algorithm

In [0]:
def floyds(f, x0):
  """
  Retrieved from https://en.wikipedia.org/wiki/Cycle_detection
  f: mapping function
  x0: initial value
  """

  tortoise = f(x0)
  hare = f(f(x0))
  while tortoise != hare:
    tortoise = f(tortoise)
    hare = f(f(hare))

  mu = 0
  tortoise = x0
  while tortoise != hare:
    tortoise = f(tortoise)
    hare = f(hare)  
    mu += 1

  lam = 1
  hare = f(tortoise)
  while tortoise != hare:
    hare = f(hare)
    lam += 1

  return lam, mu

## Problem 2

### Find Order

Order of an integer *a* modulo *n* is the smallest positive integer *r* such that:

*a*<sup>*r*</sup> = 1 mod *n*

Note that I was unable to leverage the cycle detection method to compute the order. To remedy this, I implement a linear time approach to finding the order of a modulo N.

The below method performs a linear search across all integers [1, N-1] until an integer r is found that satifies the above equation. In order to implement this in true O(n) runtime, we keep track of the previous a^r value, so that this computation can be done in constant time in each iteration.

In [0]:
def find_order(a, N, verbose=False):
  """Finds the order of a modulo N in O(N) time"""
  
  # Ensure a and n are co-prime
  if np.gcd(a, N) != 1:
    if verbose:
      print('GCD of a: {} and N: {} is not 1.'.format(a, N))
    return None

  # Current order we're trying 
  order = 1

  # Store the result of a raised to the power of order - 1
  result = 1

  # Try orders from 0 to N-1
  while order < N:
    # Calculate a^order mod n = 1
    result = (result * a) % N

    # Return current order if a^order mod n = 1
    if result == 1:
      return order
    
    # increment order
    order += 1
  if verbose:
    print('Order of {} mod {} not found'.format(a, N))
  return None

Displaying the functionality of my find_order function on 10 random a-N pairs

In [27]:
examples_so_far = 0
while examples_so_far < 10:
  a = np.random.randint(100)
  N = np.random.randint(100)
  order = find_order(a, N)
  if order is not None:
    print('Order of {} modulo {} is: {}'.format(a, N, order))
    examples_so_far += 1

Order of 89 modulo 63 is: 6
Order of 45 modulo 62 is: 15
Order of 31 modulo 63 is: 6
Order of 4 modulo 45 is: 6
Order of 26 modulo 37 is: 3
Order of 79 modulo 75 is: 10
Order of 41 modulo 43 is: 7
Order of 12 modulo 17 is: 16
Order of 59 modulo 39 is: 12
Order of 7 modulo 50 is: 4


## Problem 3

### Factoring

We can now leverage the find_order function from above to factor integers.

In [0]:
def factor(N):
    if N < 3:
      print('N must be >= 3')
      return None

    print('FACTORING {}'.format(N))
    possible_as = [i for i in range(2, N)]

    # choose a uniformly at random in {2,...,N-1}
    a = np.random.choice(possible_as)
    print('Chose a: {}'.format(a))
    possible_as.remove(a)

    # compute the order r of a modulo N using the method from problem 2
    r = find_order(a, N, True)

    # if gcd(a, N) !=  1 or order a mod n is DNE or r is odd, choose different a
    while r is None or r % 2 == 1:
      if r is not None and r % 2 == 1:
        print('order r: {} was odd, picking new a'.format(r))

      if len(possible_as) == 0:
        print('Every a has been chosen with no successful results. Returning None')
        return

      a = np.random.choice(possible_as)
      print('Chose a: {}'.format(a))
      possible_as.remove(a)

      r = find_order(a, N, True)

    # if r even, compute f = gcd(a**(r/2 -1), N))
    temp = a ** ((r / 2) - 1)
    temp_int = int(temp)
    if temp != temp_int:
      print('Error, used odd r when calculating f')
      return

    f = np.gcd(temp_int, N)

    # if f is a trivial factor, choose different a
    while f == 1 or f == -1 or f == N or f == -N:
      print('f: {} is a trivial factor, picking new a'.format(f))

      if len(possible_as) == 0:
        print('Every a has been chosen with no successful results. Returning None')
        return

      a = np.random.choice(possible_as)
      print('Chose a: {}'.format(a))
      possible_as.remove(a)

      r = find_order(a, N, True)

      while r is None or r % 2 == 1:
        if r is not None and r % 2 == 1:
          print('order r: {} was odd, picking new a'.format(r))

        if len(possible_as) == 0:
          print('Every a has been chosen with no successful results. Returning None')
          return

        a = np.random.choice(possible_as)
        print('Chose a: {}'.format(a))
        possible_as.remove(a)

        r = find_order(a, N, True)
        
      temp = a ** ((r / 2) - 1)
      temp_int = int(temp)
      if temp != temp_int:
        print('Error, used odd r when calculating f')
        return

      f = np.gcd(temp_int, N)

    # Return non trivial factor
    print('A factor of {} is {} !'.format(N, f))
    return f

Leveraging our factor() function to factor some example integers.

In [90]:
f = factor(46)

FACTORING 46
Chose a: 27
order r: 11 was odd, picking new a
Chose a: 36
GCD of a: 36 and N: 46 is not 1.
Chose a: 23
GCD of a: 23 and N: 46 is not 1.
Chose a: 7
f: 1 is a trivial factor, picking new a
Chose a: 19
f: 1 is a trivial factor, picking new a
Chose a: 2
GCD of a: 2 and N: 46 is not 1.
Chose a: 38
GCD of a: 38 and N: 46 is not 1.
Chose a: 10
GCD of a: 10 and N: 46 is not 1.
Chose a: 22
GCD of a: 22 and N: 46 is not 1.
Chose a: 35
order r: 11 was odd, picking new a
Chose a: 11
f: 1 is a trivial factor, picking new a
Chose a: 40
GCD of a: 40 and N: 46 is not 1.
Chose a: 41
order r: 11 was odd, picking new a
Chose a: 3
order r: 11 was odd, picking new a
Chose a: 39
order r: 11 was odd, picking new a
Chose a: 43
A factor of 46 is 2 !


In [84]:
f = factor(58)

FACTORING 58
Chose a: 30
GCD of a: 30 and N: 58 is not 1.
Chose a: 6
GCD of a: 6 and N: 58 is not 1.
Chose a: 56
GCD of a: 56 and N: 58 is not 1.
Chose a: 47
A factor of 58 is 2 !


In [85]:
factor(147)

FACTORING 147
Chose a: 49
GCD of a: 49 and N: 147 is not 1.
Chose a: 23
A factor of 147 is 3 !


3

In [88]:
f = factor(189)

FACTORING 189
Chose a: 120
GCD of a: 120 and N: 189 is not 1.
Chose a: 99
GCD of a: 99 and N: 189 is not 1.
Chose a: 78
GCD of a: 78 and N: 189 is not 1.
Chose a: 12
GCD of a: 12 and N: 189 is not 1.
Chose a: 167
f: 1 is a trivial factor, picking new a
Chose a: 7
GCD of a: 7 and N: 189 is not 1.
Chose a: 139
A factor of 189 is 21 !
